In [112]:
import os
import pandas as pd
import cdsapi
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import joblib
from datetime import datetime
from scipy.interpolate import interp2d
import xarray as xr
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Define MLP model
class MLP(nn.Module):
    def __init__(self, input_size, use_dropout=False, dropout_rate=0.3, use_batch_norm=False, activation_fn=nn.ReLU):
        super(MLP, self).__init__()
        layers = [
            nn.Linear(input_size, 256),
            nn.BatchNorm1d(256) if use_batch_norm else nn.Identity(),
            activation_fn(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128) if use_batch_norm else nn.Identity(),
            activation_fn(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64) if use_batch_norm else nn.Identity(),
            activation_fn(),
        ]
        if use_dropout:
            layers.append(nn.Dropout(dropout_rate))
        layers.append(nn.Linear(64, 1))
        self.model = nn.Sequential(*[layer for layer in layers if not isinstance(layer, nn.Identity)])

    def forward(self, x):
        return self.model(x)

# Load model files
model_path = "model2/"
input_size = torch.load(os.path.join(model_path, "input_size"), weights_only=True)
scalers = joblib.load(os.path.join(model_path, "scalers.pkl"))
model_params_path = os.path.join(model_path, "trained_parameters.pth")
known_turbine_types = np.load(os.path.join(model_path, "turbine_types_order.npy"))

# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_size=input_size).to(device)
model.load_state_dict(torch.load(model_params_path, weights_only=True))
model.eval()

data_dir = "crowdsourced_data/"
for file in os.listdir(data_dir):
    if file.endswith(".xlsx"):
        file_path = os.path.join(data_dir, file)
        sheets = pd.read_excel(file_path, sheet_name=["Time Series", "Metadata"])
        time_series_data = sheets["Time Series"]
        metadata = sheets["Metadata"]

        time_series_data["Date"] = pd.to_datetime(time_series_data["Date"], errors='coerce')

        years = list(set(time_series_data["Date"].dt.year.dropna().astype(str)))
        months = list(set(time_series_data["Date"].dt.month.dropna().astype(str).str.zfill(2)))
        days = list(set(time_series_data["Date"].dt.day.dropna().astype(str).str.zfill(2)))
        hours = list(set(time_series_data["Date"].dt.hour.dropna().astype(str).str.zfill(2)))

        # Prepare API request for the full period
        lat, lon = metadata.iloc[0]["Latitude"], metadata.iloc[0]["Longitude"]
        client = cdsapi.Client()
        request = {
            "product_type": "reanalysis",
            "variable": ["100m_u_component_of_wind", "100m_v_component_of_wind"],
            "year": years,
            "month": months,
            "day": days,
            "time": hours,
            "data_format": "grib",
            "area": [lat + 1, lon - 1, lat - 1, lon + 1],
        }

        # Download all data at once
        file_name = f"wind_data_{file}.grib"
        client.retrieve("reanalysis-era5-single-levels", request).download(file_name)

        # Read GRIB file using xarray and cfgrib
        ds = xr.open_dataset(file_name, engine='cfgrib')

        # Extract u and v wind components, latitudes, longitudes, and times
        u_wind = ds["u100"].values
        v_wind = ds["v100"].values
        latitudes = ds["latitude"].values
        longitudes = ds["longitude"].values
        times = pd.to_datetime(ds["time"].values)

        # Prepare static features (shared across timestamps)
        hub_height = metadata.iloc[0]["Hub Height"]
        commissioning_year = metadata.iloc[0]["Commissioning Year"]
        commissioning_month = metadata.iloc[0]["Commissioning Month"]
        ref_date = pd.Timestamp("2024-12-01")
        age_months = (ref_date.year - commissioning_year) * 12 + (ref_date.month - commissioning_month)
        capacity = metadata.iloc[0]["Capacity (MW)"]
        turbine_type = metadata.iloc[0]["Turbine Type"]

        scaled_hub_height = scalers["hub_heights"].transform([[hub_height]])[0][0]
        scaled_age = scalers["ages"].transform([[age_months]])[0][0]

        num_rows = len(time_series_data["Date"])
        hub_height_repeated = np.full((num_rows, 1), scaled_hub_height)
        age_repeated = np.full((num_rows, 1), scaled_age)

        turbine_type_encoded = np.zeros(len(known_turbine_types))
        if turbine_type in known_turbine_types:
            turbine_type_encoded[np.where(known_turbine_types == turbine_type)[0][0]] = 1
        turbine_type_repeated = np.tile(turbine_type_encoded, (num_rows, 1))

        interpolated_production = []

        # Iterate over each timestamp in the uploaded data
        for _, row in time_series_data.iterrows():
            timestamp = row["Date"]
            production_value = row["Production (kW)"] / 1e3 # convert to MW

            if timestamp in times.values:
                time_index = times.get_loc(timestamp)

                u = u_wind[time_index]
                v = v_wind[time_index]
                wind_speed = np.sqrt(u**2 + v**2)

                interpolator = interp2d(longitudes, latitudes, wind_speed, kind='linear')
                wind_speed_value = interpolator(lon, lat)[0]
                wind_speed_value = round(wind_speed_value, 2)

                interpolated_production.append([timestamp, production_value, wind_speed_value])

        productions = np.array([interpolated_value[1] for interpolated_value in interpolated_production]).reshape(-1, 1)
        wind_speeds = np.array([interpolated_value[2] for interpolated_value in interpolated_production])
        
        scaled_wind_speeds = scalers["winds"].transform(wind_speeds.reshape(-1, 1))

        # Prepare input features
        input_features = np.hstack([turbine_type_repeated, hub_height_repeated, age_repeated, scaled_wind_speeds])
        input_tensor = torch.tensor(input_features, dtype=torch.float32).to(device)

        with torch.no_grad():
            predictions = model(input_tensor)

        huber_loss = nn.HuberLoss()(predictions, torch.tensor(productions)).item()

        # Update model if the huber loss is < 1
        if huber_loss < 1:
            model.train()
            optimizer = optim.Adam(model.parameters(), lr=1e-3)
            for epoch in range(10):
                optimizer.zero_grad()
                output = model(input_tensor)
                loss = nn.HuberLoss()(output, torch.tensor(productions, dtype=torch.float32).to(device))
                loss.backward()
                optimizer.step()

            # Save updated model
            timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
            updated_model_path = os.path.join(model_path, f"trained_parameters_{timestamp_str}.pth")
            torch.save(model.state_dict(), updated_model_path)
            print(f"Updated model saved at {updated_model_path}")


2025-01-13 21:56:11,638 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2025-01-13 21:56:11,639 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-01-13 21:56:11,640 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2025-01-13 21:56:11,641 WARNING [2024-06-16T00:00:00] CDS API syntax is changed and some keys or parameter names may have also changed. To avoid requests failing, please use the "Show API request code" tool on the dataset Download Form to check you are using t

71f06b7eb4bc88ec4e084a07c992fdd8.grib:   0%|          | 0.00/88.6k [00:00<?, ?B/s]

Ignoring index file 'wind_data_time_series_20250113_160903.xlsx.grib.5b7b6.idx' older than GRIB file


Updated model saved at model2/trained_parameters_20250113_215624.pth
